# Code to create vocab.txt and answers.pkl

In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

C:\Users\miman\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\miman\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\miman\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\miman\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

### Creating vocab file

It will contain words from all questions, answers and reviews present

In [22]:
questions = pd.read_csv('Questions_Answers_126Products.csv')
reviews = pd.read_csv('Reviews_126Products.csv')

In [23]:
questions.tail()

,Prod_ID,Ques_ID,Question Text,Ans_ID,Ans_Text,Q_A
2726,125,735,Does this base fit a 2011 Graco Snugride 35 ca...,2726,Yes that's the year my snug ride was manufactu...,Does this base fit a 2011 Graco Snugride 35 ca...
2727,125,736,Does this base fit \'Graco UrbanLite Classic C...,2727,Yes it does ! Lightweight travel never looked ...,Does this base fit \'Graco UrbanLite Classic C...
2728,125,736,Does this base fit \'Graco UrbanLite Classic C...,2728,"Yes, it does. According to Graco's website des...",Does this base fit \'Graco UrbanLite Classic C...
2729,125,736,Does this base fit \'Graco UrbanLite Classic C...,2729,It does work check the track website and it ag...,Does this base fit \'Graco UrbanLite Classic C...
2730,125,736,Does this base fit \'Graco UrbanLite Classic C...,2730,The model we bought is the classic connect 35 ...,Does this base fit \'Graco UrbanLite Classic C...


In [24]:
reviews.head()

,Review_ID,Product ID,Review_Text
0,0,1,'not really as portrayed on video
1,1,1,'HORRIBLE installation experience. Installed p...
2,2,1,'I wish we'd bought the cheaper \'push latch\'...
3,3,1,"'The concept is great, but these things are a ..."
4,4,1,'My husband installed 4 of these locks on my s...


In [25]:
questions.shape, reviews.shape

((2731, 6), (27573, 3))

In [26]:
from string import punctuation
punctuation = list(punctuation)

In [27]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in punctuation:
           # pos = pos_tag([i.strip()])
            word = lemmatizer.lemmatize(i.strip())
            final_text.append(word.lower())
    return " ".join(final_text)

In [28]:
reviews = reviews.dropna()
questions = questions.dropna()

In [29]:
questions.shape, reviews.shape

((2731, 6), (27573, 3))

In [30]:
questions['Q'] = questions.Q_A.apply(lemmatize_words)
reviews['R'] = reviews.Review_Text.apply(lemmatize_words)

In [31]:
import re,string,unicodedata
questions.Q = [re.sub(r'\d+', '', x) for x in questions.Q]
reviews.R = [re.sub(r'\d+', '', x) for x in reviews.R]

In [32]:
len(questions.Q), len(reviews.R)

(2731, 27573)

In [33]:
reviews = reviews.dropna()
questions = questions.dropna()

In [34]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(questions.Q))
tokenizer.fit_on_texts(list(reviews.R))
word_index = tokenizer.word_index
#The words are given indexes based on their commonality, with the most common word being given the lowest Index

In [35]:
len(word_index)

27150

In [36]:
data = pd.DataFrame.from_dict(data=word_index, orient='index')
#data.to_csv('Vocab_file.csv')

In [37]:
data.shape

(27150, 1)

In [38]:
data.head(2)

,0
the,1
it,2


In [39]:
data = data.reset_index()

In [40]:
data.head(2)

,index,0
0,the,1
1,it,2


In [68]:
with open('vocab.txt', 'w') as filehandle:
    index = 1
    for word in list(data["index"]):
        filehandle.write("{} {}\n".format(index, word))
        index += 1
        
filehandle.close()

### Creating answers.pkl file

We use mapping file and reviews file to create answer.pkl

In [42]:
#df = pd.read_csv('mapping_formatted.csv')
rev = pd.read_csv("Reviews_126Products.csv")
#df.head()

In [43]:
#df.shape, rev.shape

In [44]:
flat_list_rev = list(range(0,27573)) ## since we have 27573 reviews in our review file

In [45]:
Reviews = rev.Review_Text
R_Copy = Reviews
new=list(R_Copy)

In [46]:
reviews_all = []
for i in flat_list_rev:
    reviews_all.append(new[i])

In [47]:
ans_embed = []
for review in reviews_all:
    sequences = tokenizer.texts_to_sequences(review)
    flat_list = []
    for sublist in sequences:
        for item in sublist:
            flat_list.append(item)
    ans_embed.append(flat_list)

In [51]:
len(ans_embed)

27573

In [52]:
myDict = {} 
  
# Creating a list 
valList = flat_list_rev
  
# Iterating the elements in list 
for val in valList: 
    for ele in ans_embed:
        type(ele)
        type(val)
        myDict[val] = ele
  

In [53]:
type(ele)

list

In [55]:
import pickle

your_dictionary = myDict

#pickle.dump(your_dictionary, open('answers_amazon.pkl', 'wb'))

In [56]:
your_dictionary

{0: [112,
  1138,
  1332,
  2681,
  2681,
  838,
  223,
  6,
  2681,
  1873,
  6,
  675,
  1214,
  1873,
  5,
  2681,
  2681,
  5,
  1873,
  6,
  1681,
  675,
  1873,
  1332,
  1806,
  2092,
  2681,
  5,
  1873,
  1332,
  1806,
  5,
  3763,
  1332,
  1214,
  6,
  675,
  1214,
  1873,
  5,
  2681,
  2681,
  1332,
  5,
  1214,
  1918,
  1214,
  1138,
  6,
  675,
  2041,
  1214,
  6,
  675,
  5,
  675,
  1718,
  1681,
  223,
  1873,
  1138,
  5370,
  6,
  861,
  5370,
  6,
  1214,
  2041,
  1234,
  1332,
  5,
  1873,
  2812,
  1681,
  1234,
  1873,
  1681,
  2092,
  1681,
  2812,
  1214,
  1873,
  5,
  6,
  1234,
  1214,
  3763,
  1332,
  1332,
  2092,
  1214,
  1873,
  5370,
  1332,
  2041,
  1234,
  5,
  675,
  1718,
  3763,
  6,
  1718,
  1214,
  861,
  1681,
  1234,
  1234,
  5,
  2681,
  1332,
  1718,
  2092,
  1332,
  1234,
  2812,
  1332,
  861,
  1873,
  2681,
  1918],
 1: [112,
  1138,
  1332,
  2681,
  2681,
  838,
  223,
  6,
  2681,
  1873,
  6,
  675,
  1214,
  1873,
  5,
  2